In [6]:
import subprocess
import os
# 로컬에서는 이미 expect가 설치되어있어야합니다. 없으면 3번째 줄 주석하시고 local_set() 함수 실행하세요
# 그 후에 다시 주석 해제해주세요.
user = "root"
host = "172.16.254.12"
password = "asd123!@"


########################### 로컬 ###########################
## 우선 로컬에서 원격 호스트 IP에 대한 StrictHostKeyChecki0 ng=no를 항상 허용해줍니다.

def local_ssh_config():
    ssh_dir = os.path.expanduser("~/.ssh")
    config_path = os.path.join(ssh_dir,"config")
    
    os.makedirs(ssh_dir, exist_ok=True)
    os.chmod(ssh_dir, 0o700)
    
    config_content = f"""
    Host {host}
        StrictHostKeyChecking no
        UserKnownHostsFile=/dev/null
    """
    
    # 이미 같은 설정이 있는지 확인하고 없으면 추가
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            content = f.read()
    else:
        content = ""
    
    if host not in content:
        with open(config_path, "a") as f:
            f.write(config_content)
        print(f"SSH 설정 추가 완료: {host}")
    else:
        print(f"이미 설정되어 있습니다: {host}")
    
    # 권한 설정 (보안상 필요)
    os.chmod(config_path, 0o600)

def local_set():
    print("로컬에서는 이미 expect가 설치되어있어야합니다. 없으면 3번째 줄 주석하시고 local_set() 함수 실행")
    os.system("dnf install -y expect")

#local_ssh_config()
#local_set()


########################### 원격 ###########################

def cc(command):
    cmd = f"sshpass -p '{password}' ssh {user}@{host} '{command}'"
    result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
    print(result.stdout)

def install_packages():
    # 여러 명령을 한 줄로 연결하기 (&& 로 순차 실행)
    commands = """
     apt -y install net-tools &&
     apt -y install vim &&
     apt -y update && apt -y upgrade
     apt -y install wget
     apt -y install expect
    """
    cc(commands)

def install_miniconda_and_config():
    print("Miniconda를 설치합니다.")
    print("/etc/bash.bashrc 파일에 Miniconda를 등록해서 , 터미널 어디서든 conda 명령을 쓸 수 있게 만드는 설정입니다.")
    commands = """
cd /tmp
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
bash miniconda.sh -b -p /root/miniconda3
echo "export PATH=\$PATH:/root/miniconda3/bin/" | sudo tee -a /etc/bash.bashrc
source /etc/bash.bashrc
"""
    cc(commands)

def install_conda_and_paramiko():
    print("conda를 설치합니다")
    commands = """
/root/miniconda3/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
/root/miniconda3/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r
/root/miniconda3/bin/conda install -y jupyter
"""
    cc(commands)
    #cc("echo 'source /etc/profile.d/conda.sh' >> /etc/bashrc")
    cc("apt install expect")
    

def set_firewalld_and_selinux_off():
    print("ufw , selinux 방화벽을 끕니다")
    cc("systemctl stop ufw")

def set_jupyter_passwd():
    
    passwd = input("주피터 패스워드 비밀번호를 설정하세요 (ex. asd123!@)")
    #cc("sudo bash -c 'source /etc/profile.d/conda.sh && /usr/bin/expect /root/jupyter_passwd.exp'")
    commands = f"""
cat << EOF > /root/jupyter_passwd.exp
#!/usr/bin/expect -f
spawn /opt/miniconda3/bin/jupyter server password
expect "Enter password"
send "{passwd}\\r"
expect "Verify password"
send "{passwd}\\r"
expect eof
EOF

chmod +x /root/jupyter_passwd.exp

sudo bash -c "/usr/bin/expect /root/jupyter_passwd.exp"
"""


    cc(commands)


# 재부팅 시 자동 활성화 설정입니다.
def set_systemd_service():
    commands = f"""
cat << EOF > /etc/systemd/system/jupyter.service
[Unit]
Description=Jupyter Notebook
After=network.target

[Service]
Type=simple
User=root
WorkingDirectory=/root
ExecStart=/root/miniconda3/bin/jupyter notebook --config=/root/.jupyter/jupyter_server_config.json --ip={host} --port=8000 --no-browser --allow-root

[Install]
WantedBy=multi-user.target
EOF

sudo systemctl daemon-reload
sudo systemctl enable jupyter
sudo systemctl start jupyter
sudo systemctl status jupyter
"""
    cc(commands)

# 실행
def install_in():
 #   install_packages()
 #   install_miniconda_and_config()   
 #   install_conda_and_paramiko()
 #   set_firewalld_and_selinux_off()
    set_jupyter_passwd()
    set_systemd_service()
    
install_in()


<>:76: SyntaxWarning: invalid escape sequence '\$'
<>:76: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipykernel_1514771/2590691846.py:76: SyntaxWarning: invalid escape sequence '\$'
  echo "export PATH=\$PATH:/root/miniconda3/bin/" | sudo tee -a /etc/bash.bashrc


conda를 설치합니다
accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r
2 channel Terms of Service accepted
Channels:
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Reading package lists...
Building dependency tree...
Reading state information...
expect is already the newest version (5.45.4-3).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.

ufw , selinux 방화벽을 끕니다



주피터 패스워드 비밀번호를 설정하세요 (ex. asd123!@) asd123!@


spawn /opt/miniconda3/bin/jupyter server password
Enter password: 
Verify password: 
[JupyterPasswordApp] Wrote hashed password to /root/.jupyter/jupyter_server_config.json

● jupyter.service - Jupyter Notebook
     Loaded: loaded (/etc/systemd/system/jupyter.service; enabled; preset: enabled)
     Active: active (running) since Mon 2025-11-24 07:07:42 UTC; 23ms ago
   Main PID: 50439 (jupyter)
      Tasks: 1 (limit: 4545)
     Memory: 2.3M (peak: 2.5M)
        CPU: 9ms
     CGroup: /system.slice/jupyter.service
             └─50439 /root/miniconda3/bin/python /root/miniconda3/bin/jupyter notebook --config=/root/.jupyter/jupyter_server_config.json --ip=172.16.254.12 --port=8000 --no-browser --allow-root

Nov 24 07:07:42 bs systemd[1]: Started jupyter.service - Jupyter Notebook.

